In [2]:
import cv2
import numpy as np
from faceie import detect_faces, encode_faces
from PIL import Image

for i in range(1, 2):
    image = Image.open("v"+str(i)+".jpeg")

    result = detect_faces(image)
    if result:
        for i in result.images:
            i.show()


In [ ]:
import cv2
import numpy as np
from faceie import detect_faces, encode_faces
from PIL import Image

known_embeddings = []
known_names = []

def register_face(image_path, name):
    image = Image.open(image_path)

    result = detect_faces(image)
    if result:
        known_embeddings.append(encode_faces(result.images))
        known_names.append(name)

for i in range(1, 6):
    register_face("p"+str(i)+".jpeg", "Fagner")

for i in range(1, 6):
    register_face("v"+str(i)+".jpeg", "Eveline")

cap = cv2.VideoCapture(0)

count = 0
while True:
    count += 1

    ret, frame = cap.read()
    if not ret:
        break

    results = None

    if(count < 5):
        continue

    count = 0
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = detect_faces(Image.fromarray(rgb_frame))

    if len(results.images)>0:
        box = results.bounding_boxes[0].astype(np.int32)
        embedding = encode_faces(results.images)

        distances = [np.linalg.norm(embedding - db_emb) for db_emb in known_embeddings]
        if len(distances) > 0:
            min_dist = np.min(distances)
            min_index = np.argmin(distances)

            if min_dist < 0.8: 
                name = known_names[min_index]
            else:
                name = "Unknown"
        else:
            name = "Unknown"

        cv2.rectangle(frame, pt1=(box[0], box[1]), pt2=(box[2], box[3]), color=(255, 0, 0), thickness=2)
        cv2.putText(frame, name, (box[0], box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    cv2.imshow('Face Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()